<a href="https://colab.research.google.com/github/carolinajacoby/farmacovigilanciaSUS/blob/main/MestradoSparkNotificacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1- Configuração do ambiente de programação



In [ ]:
import os

In [ ]:
# Instalar e testar versão do Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
! java --version

openjdk 11.0.25 2024-10-15
OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)


In [ ]:
# Baixar o Apache Spark
! wget https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
# Descompactar
! tar xf spark-3.4.4-bin-hadoop3.tgz
# Configurar o Apache Spark
! pip install -q findspark


--2025-01-08 21:48:48--  https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388988563 (371M) [application/x-gzip]
Saving to: ‘spark-3.4.4-bin-hadoop3.tgz’

spark-3.4.4-bin-had 100%[===================>] 370.97M   166MB/s    in 2.2s    

2025-01-08 21:48:55 (166 MB/s) - ‘spark-3.4.4-bin-hadoop3.tgz’ saved [388988563/388988563]



In [ ]:
#Apagar o arquivo compactado
! rm -rf spark-3.4.4-bin-hadoop3.tgz

In [ ]:
#Configuração as variáveis de ambiente
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/content/spark-3.4.4-bin-hadoop3'


In [ ]:
! pip install -q findspark # pacote para procurar o spark na máquina e informar ao os onde encontra-lo

In [ ]:
# Inicializar o Spark e criar uma sessão
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName ('MestradoSpark'). getOrCreate()

# 2- Leitura do CSV vigimed_notificacoes

In [ ]:
df_notificacoes = spark.read.csv(
    '/content/drive/MyDrive/mestrado_carol/dissertacao/codigo/vigimed/vigimed_notificacoes.csv',
    sep=';',
    header=True,
    inferSchema=True,
    encoding='ISO-8859-1'
)


In [ ]:
df_notificacoes.describe().show(20)

+-------+------+--------------------+--------------------+-------------------------+---------------------+-----------------------+-------------------+--------------------+------------------------+--------------------+--------------------+-----------+--------------------------------+------------+--------+--------+-----------------+------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+------------------+--------------------------+------------------------+--------------------+-----------+
|summary|    UF|TIPO_ENTRADA_VIGIMED|         RECEBIDO_DE|IDENTIFICACAO_NOTIFICACAO|DATA_INCLUSAO_SISTEMA|DATA_ULTIMA_ATUALIZACAO|   DATA_NOTIFICACAO|    TIPO_NOTIFICACAO|NOTIFICACAO_PARENT_CHILD|     DATA_NASCIMENTO|IDADE_MOMENTO_REACAO|GRUPO_IDADE|IDADE_GESTACIONAL_MOMENTO_REACAO|        SEXO|GESTANTE|LACTANTE|          PESO_KG|         ALTURA_CM|REACAO_EVENTO_ADVERSO_MEDDRA|               GRAVE|          

- # DIMENSÕES DA TABELA NOTIFICAÇÕES

In [ ]:
num_linhas = df_notificacoes.count()
num_colunas = len(df_notificacoes.columns)
print(f"A tabela tem {num_linhas} linhas e {num_colunas} colunas.")

df_notificacoes.printSchema()  # Esquema da tabela
df_notificacoes.show(5)        # Exibir as primeiras 5 linhas

A tabela tem 243352 linhas e 29 colunas.
root
 |-- UF: string (nullable = true)
 |-- TIPO_ENTRADA_VIGIMED: string (nullable = true)
 |-- RECEBIDO_DE: string (nullable = true)
 |-- IDENTIFICACAO_NOTIFICACAO: string (nullable = true)
 |-- DATA_INCLUSAO_SISTEMA: integer (nullable = true)
 |-- DATA_ULTIMA_ATUALIZACAO: string (nullable = true)
 |-- DATA_NOTIFICACAO: string (nullable = true)
 |-- TIPO_NOTIFICACAO: string (nullable = true)
 |-- NOTIFICACAO_PARENT_CHILD: string (nullable = true)
 |-- DATA_NASCIMENTO: string (nullable = true)
 |-- IDADE_MOMENTO_REACAO: string (nullable = true)
 |-- GRUPO_IDADE: string (nullable = true)
 |-- IDADE_GESTACIONAL_MOMENTO_REACAO: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- GESTANTE: string (nullable = true)
 |-- LACTANTE: string (nullable = true)
 |-- PESO_KG: string (nullable = true)
 |-- ALTURA_CM: integer (nullable = true)
 |-- REACAO_EVENTO_ADVERSO_MEDDRA: string (nullable = true)
 |-- GRAVE: string (nullable = true)
 |-- GR

# 3 - Pré processamento de dados

REMOÇÃO DE RUIDOS


*   ACENTOS
*   BR-ANVISA
* _x000D_


In [ ]:
from pyspark.sql.functions import regexp_replace, col, upper, to_date

# Função para remover acentos
def remover_acentos(df, colunas):
    for coluna in colunas:
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[áàãâä]", "A"))
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[éèêë]", "E"))
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[íìîï]", "I"))
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[óòõôö]", "O"))
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[úùûü]", "U"))
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[ç]", "C"))
        df = df.withColumn(coluna, regexp_replace(col(coluna), "[ñ]", "N"))
    return df



In [ ]:
# Identificar as colunas de texto
colunas_texto = [col for col, dtype in df_notificacoes.dtypes if dtype == "string"]

# Remover "BR-ANVISA-", "_x000D_", e acentos
for coluna in colunas_texto:
    df_notificacoes = df_notificacoes.withColumn(coluna, regexp_replace(col(coluna), "BR-ANVISA-", ""))
    df_notificacoes = df_notificacoes.withColumn(coluna, regexp_replace(col(coluna), "_x000D_", ""))
df_notificacoes = remover_acentos(df_notificacoes, colunas_texto)




TRANSFORMAÇÃO DE TODOS REGISTROS PARA MAIÚSCULA PARA PADRONIZAR

In [ ]:
# Transformar todos os registros para maiúsculas
for coluna in colunas_texto:
    df_notificacoes = df_notificacoes.withColumn(coluna, upper(col(coluna)))


REMOÇÃO DE ESPAÇOS EM BRANCO

In [ ]:
from pyspark.sql.functions import trim

# Remover espaços em branco de colunas de texto
colunas_texto = [
    "UF", "TIPO_ENTRADA_VIGIMED", "RECEBIDO_DE", "TIPO_NOTIFICACAO",
    "NOTIFICACAO_PARENT_CHILD", "IDADE_MOMENTO_REACAO", "GRUPO_IDADE",
    "IDADE_GESTACIONAL_MOMENTO_REACAO", "REACAO_EVENTO_ADVERSO_MEDDRA",
    "GRAVIDADE", "DESFECHO", "DURACAO", "RELACAO_MEDICAMENTO_EVENTO",
    "NOME_MEDICAMENTO_WHODRUG", "ACAO_ADOTADA", "NOTIFICADOR"
]

# Aplicar trim para remover espaços em branco no início e no fim
for coluna in colunas_texto:
    df_notificacoes = df_notificacoes.withColumn(coluna, trim(col(coluna)))


VERIFICAR VALORES AUSENTES OU NULOS

In [ ]:
from pyspark.sql.functions import col, isnan, when, count

# Número total de linhas no DataFrame
total_rows = df_notificacoes.count()

# Calcular valores ausentes (nulos ou NaN) e sua porcentagem para cada coluna
missing_values = df_notificacoes.select([
    count(when(col(c).isNull() | isnan(col(c)), c)).alias(c + "_missing_count") for c in df_notificacoes.columns
] + [
    (count(when(col(c).isNull() | isnan(col(c)), c)) / total_rows * 100).alias(c + "_missing_percentage") for c in df_notificacoes.columns
])

# Mostrar o resultado
missing_values.show()


+----------------+----------------------------------+-------------------------+---------------------------------------+-----------------------------------+-------------------------------------+------------------------------+------------------------------+--------------------------------------+-----------------------------+----------------------------------+-------------------------+----------------------------------------------+------------------+----------------------+----------------------+---------------------+-----------------------+------------------------------------------+-------------------+-----------------------+----------------------+------------------------------+-----------------------------+---------------------+----------------------------------------+--------------------------------------+--------------------------+-------------------------+---------------------+---------------------------------------+------------------------------+--------------------------------------

| Coluna                                      | Valores Ausentes (Contagem) | Valores Ausentes (%)      |
|---------------------------------------------|-----------------------------|----------------------------|
| UF                                          | 79153                       | 32.53                     |
| TIPO_ENTRADA_VIGIMED                        | 70                          | 0.03                      |
| RECEBIDO_DE                                 | 26702                       | 10.97                     |
| IDENTIFICACAO_NOTIFICACAO                   | 0                           | 0.00                      |
| DATA_INCLUSAO_SISTEMA                       | 0                           | 0.00                      |
| DATA_ULTIMA_ATUALIZACAO                     | 29                          | 0.01                      |
| DATA_NOTIFICACAO                            | 73092                       | 30.04                     |
| TIPO_NOTIFICACAO                            | 0                           | 0.00                      |
| NOTIFICACAO_PARENT_CHILD                    | 242408                      | 99.61                     |
| DATA_NASCIMENTO                             | 42820                       | 17.60                     |
| IDADE_MOMENTO_REACAO                        | 65484                       | 26.91                     |
| GRUPO_IDADE                                 | 99966                       | 41.08                     |
| IDADE_GESTACIONAL_MOMENTO_REACAO            | 161276                      | 66.27                     |
| SEXO                                        | 5738                        | 2.36                      |
| GESTANTE                                    | 160858                      | 66.10                     |
| LACTANTE                                    | 160861                      | 66.10                     |
| PESO_KG                                     | 94713                       | 38.92                     |
| ALTURA_CM                                   | 192269                      | 79.01                     |
| REACAO_EVENTO_ADVERSO_MEDDRA                | 197                         | 0.08                      |
| GRAVE                                       | 32983                       | 13.55                     |
| GRAVIDADE                                   | 77425                       | 31.82                     |
| DESFECHO                                    | 25033                       | 10.29                     |
| DATA_INICIO_HORA                            | 1508                        | 0.62                      |
| DATA_FINAL_HORA                             | 3069                        | 1.26                      |
| DURACAO                                     | 3784                        | 1.55                      |
| RELACAO_MEDICAMENTO_EVENTO                  | 1                           | 0.00                      |
| NOME_MEDICAMENTO_WHODRUG                    | 375                         | 0.15                      |
| ACAO_ADOTADA                                | 82834                       | 34.04                     |
| NOTIFICADOR                                 | 10306                       | 4.24                      |


## 3.1 EXCLUSÃO DE ATRIBUTOS  COM MAIS DE 75% DE VALORES FALTANTES

In [ ]:
from pyspark.sql.functions import col, isnan, when, count

# Número total de linhas no DataFrame
total_rows = df_notificacoes.count()

# Identificar as colunas com mais de 75% de valores ausentes
columns_to_keep = [
    c for c in df_notificacoes.columns
    if df_notificacoes.select(
        (count(when(col(c).isNull() | isnan(col(c)), c)) / total_rows * 100).alias("missing_percentage")
    ).first()["missing_percentage"] <= 75
]

# Criar um novo DataFrame sem as colunas com mais de 75% de valores ausentes
df_cleaned = df_notificacoes.select(columns_to_keep)

# Mostrar as colunas restantes
print("Colunas mantidas:", df_cleaned.columns)


Colunas mantidas: ['UF', 'TIPO_ENTRADA_VIGIMED', 'RECEBIDO_DE', 'IDENTIFICACAO_NOTIFICACAO', 'DATA_INCLUSAO_SISTEMA', 'DATA_ULTIMA_ATUALIZACAO', 'DATA_NOTIFICACAO', 'TIPO_NOTIFICACAO', 'DATA_NASCIMENTO', 'IDADE_MOMENTO_REACAO', 'GRUPO_IDADE', 'IDADE_GESTACIONAL_MOMENTO_REACAO', 'SEXO', 'GESTANTE', 'LACTANTE', 'PESO_KG', 'REACAO_EVENTO_ADVERSO_MEDDRA', 'GRAVE', 'GRAVIDADE', 'DESFECHO', 'DATA_INICIO_HORA', 'DATA_FINAL_HORA', 'DURACAO', 'RELACAO_MEDICAMENTO_EVENTO', 'NOME_MEDICAMENTO_WHODRUG', 'ACAO_ADOTADA', 'NOTIFICADOR']


SALVAMENTO DE DF COM AS COLUNAS EXCLUÍDAS.

Continuarei o tratamento das colunas utilizando o novo df, salvo em parquet(otimização computacional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path = '/content/drive/MyDrive/mestrado_carol/dissertacao/codigo/vigimed/df_notificacoescleaned'


In [ ]:
df_cleaned.write.mode("overwrite").parquet(save_path + ".parquet")


In [ ]:
# Caminho do arquivo salvo
save_path = '/content/drive/MyDrive/mestrado_carol/dissertacao/codigo/vigimed/df_notificacoescleaned.parquet'

# Ler o arquivo Parquet salvo como um novo DataFrame
df_cleaned_loaded = spark.read.parquet(save_path)

# Mostrar as primeiras linhas para verificar
df_cleaned_loaded.show()


+---+--------------------+--------------------+-------------------------+---------------------+-----------------------+----------------+--------------------+---------------+--------------------+-----------+--------------------------------+---------+--------+--------+-------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+------------------------+--------------------+--------------------+
| UF|TIPO_ENTRADA_VIGIMED|         RECEBIDO_DE|IDENTIFICACAO_NOTIFICACAO|DATA_INCLUSAO_SISTEMA|DATA_ULTIMA_ATUALIZACAO|DATA_NOTIFICACAO|    TIPO_NOTIFICACAO|DATA_NASCIMENTO|IDADE_MOMENTO_REACAO|GRUPO_IDADE|IDADE_GESTACIONAL_MOMENTO_REACAO|     SEXO|GESTANTE|LACTANTE|PESO_KG|REACAO_EVENTO_ADVERSO_MEDDRA|               GRAVE|           GRAVIDADE|            DESFECHO|    DATA_INICIO_HORA|     DATA_FINAL_HORA|             DURACAO|RELACAO_MEDICAMENTO_EVENTO|NOME_MEDICAMENTO_W

## 3.1 TRATAMENTO ATRIBUTO RS
Substituição de ausentes por "DESCONHECIDO"

In [ ]:
df_cleaned_loaded = df_cleaned_loaded.fillna({"UF": "DESCONHECIDO"})

# Verificar o resultado
df_cleaned_loaded.select("UF").show()


+---+
| UF|
+---+
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
| SP|
+---+
only showing top 20 rows



CONFIRMAÇÃO DE TRATAMENTO

In [ ]:
from pyspark.sql.functions import col, isnan, when, count

# Calcular a porcentagem de valores ausentes na coluna UF
total_rows = df_cleaned_loaded.count()
missing_uf_count = df_cleaned_loaded.filter(col("UF").isNull() | isnan(col("UF"))).count()
missing_uf_percentage = (missing_uf_count / total_rows) * 100

# Exibir os resultados
print(f"Total de linhas: {total_rows}")
print(f"Valores ausentes na coluna 'UF': {missing_uf_count}")
print(f"Porcentagem de valores ausentes em 'UF': {missing_uf_percentage:.2f}%")


Total de linhas: 243352
Valores ausentes na coluna 'UF': 0
Porcentagem de valores ausentes em 'UF': 0.00%


## 3.2 TRATAMENTO COLUNA TIPO_ENTRADA_VIGIMED
Como o valor é insifnificante, decidi excluir os registros ausentes.

In [ ]:
df_cleaned_loaded = df_cleaned_loaded.filter(df_cleaned_loaded.TIPO_ENTRADA_VIGIMED.isNotNull())

# Verificar o resultado
df_cleaned_loaded.select("TIPO_ENTRADA_VIGIMED").show()


+--------------------+
|TIPO_ENTRADA_VIGIMED|
+--------------------+
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
|EMPRESAS FARMACEU...|
+--------------------+
only showing top 20 rows



## 3.3 TRATAMENTO COLUNA RECEBIDO_DE

In [ ]:
df_cleaned_loaded = df_cleaned_loaded.fillna({"RECEBIDO_DE": "DESCONHECIDO"})

# Verificar o resultado
df_cleaned_loaded.select("RECEBIDO_DE").show()


+--------------------+
|         RECEBIDO_DE|
+--------------------+
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
|EMPRESA FARMACEUTICA|
+--------------------+
only showing top 20 rows



## 3.4 TRATAMENTO COLUNA DATA_ULTIMA_ATUALIZACAO
Substitui nulos por uma data padrão "1900-01-01"

In [37]:
from pyspark.sql.functions import lit

# Preencher valores ausentes com uma data padrão
df_cleaned_loaded = df_cleaned_loaded.fillna({"DATA_ULTIMA_ATUALIZACAO": "1900-01-01"})

# Verificar o resultado
df_cleaned_loaded.select("DATA_ULTIMA_ATUALIZACAO").show()


+-----------------------+
|DATA_ULTIMA_ATUALIZACAO|
+-----------------------+
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
+-----------------------+
only showing top 20 rows



ALTERAR PARA TIMESTAMP

In [38]:
from pyspark.sql.functions import to_timestamp

# Converter a coluna DATA_ULTIMA_ATUALIZACAO para o tipo Timestamp
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "DATA_ULTIMA_ATUALIZACAO",
    to_timestamp("DATA_ULTIMA_ATUALIZACAO", "yyyy-MM-dd HH:mm:ss")
)

# Verificar a atualização
df_cleaned_loaded.select("DATA_ULTIMA_ATUALIZACAO").printSchema()
df_cleaned_loaded.select("DATA_ULTIMA_ATUALIZACAO").show()


root
 |-- DATA_ULTIMA_ATUALIZACAO: timestamp (nullable = true)

+-----------------------+
|DATA_ULTIMA_ATUALIZACAO|
+-----------------------+
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
|    1900-01-01 00:00:00|
+-----------------------+
only showing top 20 rows



## 3.5 TRATAMENTO COLUNA DATA_NOTIFICACAO

In [ ]:
df_cleaned_loaded = df_cleaned_loaded.filter(col("DATA_NOTIFICACAO").isNotNull())

# Verificar o resultado
df_cleaned_loaded.select("DATA_NOTIFICACAO").show()


+----------------+
|DATA_NOTIFICACAO|
+----------------+
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
|            NONE|
+----------------+
only showing top 20 rows



ALTERAR PARA TIMESTAMP

In [ ]:
from pyspark.sql.functions import to_timestamp

# Converter a coluna DATA_NITIFICACAO para o tipo Timestamp
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "DATA_NOTIFICACAO",
    to_timestamp("DATA_NOTIFICACAO", "yyyy-MM-dd HH:mm:ss")
)

# Verificar a atualização
df_cleaned_loaded.select("DATA_NOTIFICACAO").printSchema()
df_cleaned_loaded.select("DATA_NOTIFICACAO").show()


## 3.6 TRATAMENTO COLUNA DATA_NASCIMENTO

Substitui por uma data padrão "1900-01-01"

In [ ]:
df_cleaned_loaded = df_cleaned_loaded.fillna({"DATA_NASCIMENTO": "1900-01-01"})

# Verificar o resultado
df_cleaned_loaded.select("DATA_NASCIMENTO").show()



+---------------+
|DATA_NASCIMENTO|
+---------------+
|       19900131|
|           NONE|
|       19710522|
|           NONE|
|       19820505|
|       19781005|
|       19620608|
|           NONE|
|       19600904|
|       19600111|
|       19720328|
|       19730228|
|       20200327|
|           NONE|
|       19721127|
|           NONE|
|       19790519|
|       19701026|
|       20111126|
|       19700818|
+---------------+
only showing top 20 rows



In [42]:
from pyspark.sql.functions import to_date

# Converter a coluna DATA_NASCIMENTO para o tipo Date
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "DATA_NASCIMENTO",
    to_date("DATA_NASCIMENTO", "yyyy-MM-dd")  # Ajuste o formato se necessário
)

# Verificar a atualização
df_cleaned_loaded.select("DATA_NASCIMENTO").printSchema()
df_cleaned_loaded.select("DATA_NASCIMENTO").show()


root
 |-- DATA_NASCIMENTO: date (nullable = true)

+---------------+
|DATA_NASCIMENTO|
+---------------+
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
|           null|
+---------------+
only showing top 20 rows



## 3.7 TRATAMENTO COLUNA IDADE_MOMENTO_REACAO

In [43]:
from pyspark.sql.functions import regexp_extract

# Extrair apenas os números da coluna IDADE_MOMENTO_REACAO
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "IDADE_MOMENTO_REACAO",
    regexp_extract("IDADE_MOMENTO_REACAO", r"(\d+)", 1)
)


CONVERTER PARA O TIPO INTEIRO

In [44]:
# Converter a coluna para o tipo inteiro
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "IDADE_MOMENTO_REACAO",
    col("IDADE_MOMENTO_REACAO").cast("int")
)


INPUTAR VALORES FALTANTES COM BASE EM DATA_NASCIMENTO e DATA_NOTIFICACAO

In [45]:
from pyspark.sql.functions import datediff, floor

# Imputar idade com base em DATA_NASCIMENTO e DATA_NOTIFICACAO
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "IDADE_MOMENTO_REACAO",
    when(
        col("IDADE_MOMENTO_REACAO").isNull() & col("DATA_NASCIMENTO").isNotNull() & col("DATA_NOTIFICACAO").isNotNull(),
        floor(datediff(col("DATA_NOTIFICACAO"), col("DATA_NASCIMENTO")) / 365.25)
    ).otherwise(col("IDADE_MOMENTO_REACAO"))
)


In [46]:
# Verificar o tipo da coluna e os dados
df_cleaned_loaded.select("IDADE_MOMENTO_REACAO").printSchema()
df_cleaned_loaded.select("IDADE_MOMENTO_REACAO").show()


root
 |-- IDADE_MOMENTO_REACAO: long (nullable = true)

+--------------------+
|IDADE_MOMENTO_REACAO|
+--------------------+
|                  30|
|                null|
|                  49|
|                null|
|                  39|
|                  42|
|                  61|
|                  81|
|                  60|
|                  61|
|                  49|
|                  48|
|                   1|
|                  71|
|                  48|
|                  36|
|                  41|
|                  53|
|                  10|
|                  51|
+--------------------+
only showing top 20 rows



## 3.8 TRATAMENTO COLUNA GRUPO_IDADE

In [47]:
from pyspark.sql.functions import when

# Criar grupos etários com base em IDADE_MOMENTO_REACAO
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "GRUPO_IDADE",
    when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO") < 18), "Menor de 18")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(18, 24)), "18-24 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(25, 34)), "25-34 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(35, 44)), "35-44 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(45, 54)), "45-54 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(55, 64)), "55-64 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO") >= 65), "65 anos ou mais")
    .otherwise(col("GRUPO_IDADE"))
)

# Verificar o resultado
df_cleaned_loaded.select("IDADE_MOMENTO_REACAO", "GRUPO_IDADE").show()


+--------------------+-----------+
|IDADE_MOMENTO_REACAO|GRUPO_IDADE|
+--------------------+-----------+
|                  30|       NONE|
|                null|       NONE|
|                  49|       NONE|
|                null|       NONE|
|                  39|       NONE|
|                  42|       NONE|
|                  61|       NONE|
|                  81|       NONE|
|                  60|       NONE|
|                  61|       NONE|
|                  49|       NONE|
|                  48|       NONE|
|                   1|       NONE|
|                  71|       NONE|
|                  48|       NONE|
|                  36|       NONE|
|                  41|       NONE|
|                  53|       NONE|
|                  10|       NONE|
|                  51|       NONE|
+--------------------+-----------+
only showing top 20 rows



In [48]:
# Exibir valores únicos no atributo GRUPO_IDADE
df_cleaned_loaded.select("GRUPO_IDADE").distinct().show()


+---------------+
|    GRUPO_IDADE|
+---------------+
|     18-24 anos|
|     45-54 anos|
|        NEONATO|
|    Menor de 18|
|           null|
|     25-34 anos|
|       INFANTIL|
|           FETO|
|     35-44 anos|
|         ADULTO|
|     55-64 anos|
|65 anos ou mais|
|          IDOSO|
|        CRIANCA|
|           NONE|
|    ADOLESCENTE|
+---------------+



In [49]:
from pyspark.sql.functions import when, col

# Ajustar os valores da coluna GRUPO_IDADE
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "GRUPO_IDADE",
    when(col("GRUPO_IDADE").isin("NEONATO", "INFANTIL", "FETO", "CRIANCA", "ADOLESCENTE", "Menor de 18"), "Menor de 18")
    .when(col("GRUPO_IDADE") == "18-24 anos", "18-24 anos")
    .when(col("GRUPO_IDADE") == "25-34 anos", "25-34 anos")
    .when(col("GRUPO_IDADE") == "35-44 anos", "35-44 anos")
    .when(col("GRUPO_IDADE") == "45-54 anos", "45-54 anos")
    .when(col("GRUPO_IDADE") == "55-64 anos", "55-64 anos")
    .when(col("GRUPO_IDADE").isin("IDOSO", "65 anos ou mais"), "65 anos ou mais")
    .when(col("GRUPO_IDADE").isin("null", "NONE"), "Não informado")
    .otherwise("Não informado")  # Valores não reconhecidos
)

# Verificar o resultado
df_cleaned_loaded.groupBy("GRUPO_IDADE").count().orderBy("count", ascending=False).show()


+---------------+------+
|    GRUPO_IDADE| count|
+---------------+------+
|  Não informado|121944|
|65 anos ou mais| 28955|
|    Menor de 18|  8805|
|     35-44 anos|  2696|
|     25-34 anos|  2446|
|     45-54 anos|  1977|
|     55-64 anos|  1815|
|     18-24 anos|  1606|
+---------------+------+



Tentativa de inputação de valores usando as colunas IDADE_MOMENTO_REACAO

Criação do GRUPO_IDADE com base em IDADE_MOMENTO_REACAO

In [50]:
from pyspark.sql.functions import when

# Imputar valores de GRUPO_IDADE com base em IDADE_MOMENTO_REACAO
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "GRUPO_IDADE",
    when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO") < 18), "Menor de 18")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(18, 24)), "18-24 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(25, 34)), "25-34 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(35, 44)), "35-44 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(45, 54)), "45-54 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO").between(55, 64)), "55-64 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_MOMENTO_REACAO") >= 65), "65 anos ou mais")
    .otherwise(col("GRUPO_IDADE"))
)

# Verificar o resultado
df_cleaned_loaded.groupBy("GRUPO_IDADE").count().orderBy("count", ascending=False).show()


+---------------+------+
|    GRUPO_IDADE| count|
+---------------+------+
|  Não informado|121944|
|65 anos ou mais| 28955|
|    Menor de 18|  8805|
|     35-44 anos|  2696|
|     25-34 anos|  2446|
|     45-54 anos|  1977|
|     55-64 anos|  1815|
|     18-24 anos|  1606|
+---------------+------+



Tentativa de inputação de valores usando as colunas DATA_NASCIMENTO e DATA_NOTIFICACAO

Criação do GRUPO_IDADE com base em IDADE_MOMENTO_REACAO

In [51]:
from pyspark.sql.functions import datediff, floor

# Calcular idade com base em DATA_NASCIMENTO e DATA_NOTIFICACAO
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "IDADE_CALCULADA",
    when(
        col("IDADE_MOMENTO_REACAO").isNull() & col("DATA_NASCIMENTO").isNotNull() & col("DATA_NOTIFICACAO").isNotNull(),
        floor(datediff(col("DATA_NOTIFICACAO"), col("DATA_NASCIMENTO")) / 365.25)
    ).otherwise(col("IDADE_MOMENTO_REACAO"))
)

# Imputar valores de GRUPO_IDADE com base em IDADE_CALCULADA
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "GRUPO_IDADE",
    when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA") < 18), "Menor de 18")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA").between(18, 24)), "18-24 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA").between(25, 34)), "25-34 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA").between(35, 44)), "35-44 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA").between(45, 54)), "45-54 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA").between(55, 64)), "55-64 anos")
    .when(col("GRUPO_IDADE").isNull() & (col("IDADE_CALCULADA") >= 65), "65 anos ou mais")
    .otherwise(col("GRUPO_IDADE"))
)

# Verificar o resultado
df_cleaned_loaded.groupBy("GRUPO_IDADE").count().orderBy("count", ascending=False).show()


+---------------+------+
|    GRUPO_IDADE| count|
+---------------+------+
|  Não informado|121944|
|65 anos ou mais| 28955|
|    Menor de 18|  8805|
|     35-44 anos|  2696|
|     25-34 anos|  2446|
|     45-54 anos|  1977|
|     55-64 anos|  1815|
|     18-24 anos|  1606|
+---------------+------+



In [52]:
# Substituir valores ausentes por "Não informado"
df_cleaned_loaded = df_cleaned_loaded.fillna({"GRUPO_IDADE": "NÃO INFORMADO"})

# Verificar o resultado
df_cleaned_loaded.groupBy("GRUPO_IDADE").count().orderBy("count", ascending=False).show()


+---------------+------+
|    GRUPO_IDADE| count|
+---------------+------+
|  Não informado|121944|
|65 anos ou mais| 28955|
|    Menor de 18|  8805|
|     35-44 anos|  2696|
|     25-34 anos|  2446|
|     45-54 anos|  1977|
|     55-64 anos|  1815|
|     18-24 anos|  1606|
+---------------+------+



## 3.9 TRATAMENTO DA COLUNA IDADE_GESTACIONAL_MOMENTO_REACAO

In [53]:
# Verificar valores únicos na coluna
df_cleaned_loaded.select("IDADE_GESTACIONAL_MOMENTO_REACAO").distinct().show()

# Verificar relação com SEXO
df_cleaned_loaded.groupBy("SEXO", "IDADE_GESTACIONAL_MOMENTO_REACAO").count().show()


+--------------------------------+
|IDADE_GESTACIONAL_MOMENTO_REACAO|
+--------------------------------+
|                      35 SEMANAS|
|                       7 SEMANAS|
|                      36 SEMANAS|
|                            null|
|                      24 SEMANAS|
|                      33 SEMANAS|
|                      30 SEMANAS|
|                       8 SEMANAS|
|                      38 SEMANAS|
|                       9 SEMANAS|
|                      27 SEMANAS|
|                            NONE|
|                      34 SEMANAS|
|                      21 SEMANAS|
|                      26 SEMANAS|
|                      37 SEMANAS|
|                           2 MES|
|                      25 SEMANAS|
|                      15 SEMANAS|
|                      16 SEMANAS|
+--------------------------------+
only showing top 20 rows

+------------+--------------------------------+-----+
|        SEXO|IDADE_GESTACIONAL_MOMENTO_REACAO|count|
+------------+------------

Extrair números e padronizar para semanas

In [54]:
from pyspark.sql.functions import regexp_extract, when

# Extrair números e converter meses para semanas
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "IDADE_GESTACIONAL_MOMENTO_REACAO",
    when(
        col("IDADE_GESTACIONAL_MOMENTO_REACAO").rlike(r"\d+ SEMANAS"),  # Para casos "X SEMANAS"
        regexp_extract(col("IDADE_GESTACIONAL_MOMENTO_REACAO"), r"(\d+)", 1).cast("int")
    ).when(
        col("IDADE_GESTACIONAL_MOMENTO_REACAO").rlike(r"\d+ MES"),  # Para casos "X MES"
        (regexp_extract(col("IDADE_GESTACIONAL_MOMENTO_REACAO"), r"(\d+)", 1).cast("int") * 4)
    ).otherwise(None)  # Manter nulos para tratamento posterior
)

# Verificar o resultado da padronização
df_cleaned_loaded.select("IDADE_GESTACIONAL_MOMENTO_REACAO").show()


+--------------------------------+
|IDADE_GESTACIONAL_MOMENTO_REACAO|
+--------------------------------+
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
|                            null|
+--------------------------------+
only showing top 20 rows



Tratamento de ausentes - substituir por -1 PARA MELHOR PERFORMANCE NO ML

In [55]:
df_cleaned_loaded = df_cleaned_loaded.fillna({"IDADE_GESTACIONAL_MOMENTO_REACAO": -1})

# Verificar valores padronizados
df_cleaned_loaded.select("IDADE_GESTACIONAL_MOMENTO_REACAO").show()


+--------------------------------+
|IDADE_GESTACIONAL_MOMENTO_REACAO|
+--------------------------------+
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
|                              -1|
+--------------------------------+
only showing top 20 rows



## 4.0 TRATAMENTO COLUNA SEXO

In [67]:
# Excluir linhas onde SEXO está ausente (nulo)
df_cleaned_loaded = df_cleaned_loaded.filter(col("SEXO").isNotNull())

# Verificar o resultado
df_cleaned_loaded.groupBy("SEXO").count().show()


+-------------+------+
|         SEXO| count|
+-------------+------+
|Não informado|  3552|
|    MASCULINO| 63341|
|     FEMININO|100959|
+-------------+------+



In [68]:
from pyspark.sql.functions import when, col

# Substituir "NONE" e "DESCONHECIDO" por "NÃO INFORMADO"
df_cleaned_loaded = df_cleaned_loaded.withColumn(
    "SEXO",
    when(col("SEXO").isin("NONE", "DESCONHECIDO"), "NÃO INFORMADO").otherwise(col("SEXO"))
)

# Verificar os valores únicos e suas frequências
df_cleaned_loaded.groupBy("SEXO").count().orderBy("count", ascending=False).show()


+-------------+------+
|         SEXO| count|
+-------------+------+
|     FEMININO|100959|
|    MASCULINO| 63341|
|Não informado|  3552|
+-------------+------+



TRANSFORMAÇÃO EM VARIÁVEL CATEGORICA

In [ ]:
from pyspark.ml.feature import StringIndexer

# Aplicar StringIndexer para codificar a coluna SEXO
indexer = StringIndexer(inputCol="SEXO", outputCol="SEXO_Indexed")
df_cleaned_loaded = indexer.fit(df_cleaned_loaded).transform(df_cleaned_loaded)

# Verificar o resultado
df_cleaned_loaded.select("SEXO", "SEXO_Indexed").show()


In [ ]:
from pyspark.ml.feature import OneHotEncoder

# Aplicar OneHotEncoder após a indexação
encoder = OneHotEncoder(inputCol="SEXO_Indexed", outputCol="SEXO_OneHot")
df_cleaned_loaded = encoder.fit(df_cleaned_loaded).transform(df_cleaned_loaded)

# Verificar o resultado
df_cleaned_loaded.select("SEXO", "SEXO_Indexed", "SEXO_OneHot").show()


In [ ]:
# Exibir o schema para confirmar as transformações
df_cleaned_loaded.printSchema()

# Verificar os dados
df_cleaned_loaded.select("SEXO", "SEXO_Indexed", "SEXO_OneHot").show()
